## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette
import time

In [3]:
url = 'http://192.168.1.5:5000'
volume = {'volume':5}
r = requests.post(url + '/get_status', json = {'name':'1cc_2'})
r.content

b'{"remaining_volume":null,"syringe_loaded":false}\n'

In [4]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False) 

In [5]:
#jubilee.home_all()

In [6]:
deck = jubilee.load_deck('lab_automation_deck_AFL_bolton.json')

## Load labware


In [7]:
samples = jubilee.load_labware('septavialrev1_44_holder_2000ul.json', 2)
samples.manual_offset([(19.8,178.1),(132.5,177.6),(132.5,106.6)])

New manual offset applied to septavialrev1_44_holder_2000ul


In [8]:
stocks = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks.manual_offset([(31.3,268.1),(117.6,267),(117.9,210.9)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [9]:
tiprack = jubilee.load_labware('opentrons_96_tiprack_300ul.json', 0)
tiprack.manual_offset([(28,77.3),(127.1,76.8),(127.8,14.5)])

New manual offset applied to opentrons_96_tiprack_300ul


In [10]:
trash = jubilee.load_labware('agilent_1_reservoir_290ml.json', 1)

## Load Tools

In [11]:
P300 = Pipette.Pipette.from_config(0, 'Pipette', 'P300_config.json')
jubilee.load_tool(P300)
P300.add_tiprack(tiprack)
P300.trash = trash[0]

In [12]:
syringe_10 = syringe.HTTPSyringe.from_config(1, "src/science_jubilee/tools/configs/10cc_syringe.json")
syringe_1_1 = syringe.HTTPSyringe.from_config(2, "src/science_jubilee/tools/configs/1cc_1_syringe.json")
syringe_1_2 = syringe.HTTPSyringe.from_config(3, "src/science_jubilee/tools/configs/1cc_2_syringe.json")
syringe_1_3 = syringe.HTTPSyringe.from_config(4, "src/science_jubilee/tools/configs/1cc_3_syringe.json")

In [13]:
jubilee.load_tool(syringe_10)
jubilee.load_tool(syringe_1_1)
jubilee.load_tool(syringe_1_2)
jubilee.load_tool(syringe_1_3)

In [14]:
mix_syringe = syringe_10
water_syringe = syringe_1_1
ammonia_syringe = syringe_1_2
teos_syringe = syringe_1_3

## Manual load solution

In [15]:
jubilee.pickup_tool(water_syringe)

In [29]:
teos_syringe.set_pulsewidth(1844)
teos_syringe.load_syringe(0, 1844)

Loaded syringe, remaining volume 0 uL


In [17]:
#water_syringe.set_pulsewidth(water_syringe.empty_position-1)

In [30]:
aspirate = samples[0].bottom(+200)
aspirate_vol = 995 #- syringe_1_1.remaining_volume
#aspirate_vol = 9995
teos_syringe.aspirate(aspirate_vol, aspirate)

In [19]:
jubilee.park_tool()

In [ ]:
# deck is at lowest point now

In [ ]:
# repeat for other solutions

In [30]:
#jubilee.move(dz = 50)

## Experiment

##### EtOH: 1851.8 ul
##### NH3OH: 67.6 ul
##### H20: 36 ul
##### TEOS: 44.6 ul

In [34]:
etoh_dispense_vol_1 = 300
etoh__dispense_vol_2 = 175.9
ammonia_dispense_vol = 67.6
water_dispense_vol = 36
teos_dispense_vol = 44.6
mix_vol = 2000

### Add EtOH

In [116]:
jubilee.pickup_tool(P300)

In [117]:
P300.pickup_tip()

In [60]:
#jubilee.move_to(x = samples[0].x, y = samples[0].y)

In [66]:
jubilee.move(dz = 60)

In [118]:
etoh_vol_1 = 300
etoh_vol_2 = 175.9
for i in range(2):
    for k in range(5):
        P300.aspirate(etoh_dispense_vol_1, stocks[0])
        jubilee.move(dz=60)
        P300.dispense(etoh_dispense_vol_1, samples[i].bottom(+15))
        jubilee.move(dz=60)
        time.sleep(1)
    for j in range (2):
        P300.aspirate(etoh_dispense_vol_2, stocks[0])
        jubilee.move(dz=60)
        P300.dispense(etoh_dispense_vol_2, samples[i].bottom(+15))
        jubilee.move(dz=60)
        time.sleep(1)

In [119]:
P300.drop_tip()


In [120]:
jubilee.park_tool()

### Add NH3OH (syringe_1_2)

In [94]:
jubilee.pickup_tool(ammonia_syringe)

In [95]:
#jubilee.move_to(x = samples[0].x, y = samples[0].y)

In [96]:
ammonia_dispense_vol = 67.6


for i in range(2):
    dispense_location = samples[i].bottom(+25)
    ammonia_syringe.dispense(ammonia_dispense_vol, dispense_location)
    time.sleep(3)
    #jubilee.move(dz = 20)

In [97]:
jubilee.park_tool()

### Add water (syringe_1_1)

In [99]:
jubilee.pickup_tool(water_syringe)

In [114]:
water_dispense_vol = 36


for i in range(2):
    dispense_location = samples[i].bottom(+25)
    water_syringe.dispense(water_dispense_vol, dispense_location)
    time.sleep(3)

In [115]:
jubilee.park_tool()

### Mix (syringe_10)

In [20]:
jubilee.pickup_tool(mix_syringe)

In [23]:
mix_syringe.aspirate(5, samples[0].bottom(+5))

In [26]:
for i in range(2):
    mix_loc = samples[i].bottom(+5)
    mix_syringe.mix(mix_vol, 5, mix_loc, t_hold = 3)

    wash_loc_1 = stocks[1]
    mix_syringe.aspirate(5, wash_loc_1)
    for k in range(2):
        wash_loc = stocks[k+1]
        mix_syringe.mix(mix_vol, 3, wash_loc, t_hold = 3)

aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense


In [27]:
jubilee.park_tool()

### Add TEOS and mix

In [35]:
teos_dispense_vol = 44.6


for i in range(2):
    jubilee.pickup_tool(teos_syringe)
    dispense_location = samples[i].bottom(+25)
    mix_location = samples[i].bottom(+5)
    teos_syringe.dispense(teos_dispense_vol, dispense_location)
    time.sleep(3)
    jubilee.park_tool()
    time.sleep(1)
    
    jubilee.pickup_tool(mix_syringe)
    
    mix_loc = samples[i].bottom(+5)
    mix_syringe.aspirate(5, mix_loc)
    mix_syringe.mix(mix_vol, 5, mix_loc, t_hold = 3)

    wash_loc_1 = stocks[1]
    mix_syringe.aspirate(5, wash_loc_1)
    for k in range(2):
        wash_loc = stocks[k+1]
        mix_syringe.mix(mix_vol, 3, wash_loc, t_hold = 3)

    jubilee.park_tool()
    

aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
aspirate
dispense
